<table style="width:100%">
  <tr>
    <td><center style="font-size:500%;">Representação Distribuida de Palavras (Parte 1)</center></td>
    <td><img src="https://logodownload.org/wp-content/uploads/2015/02/puc-rio-logo.gif" width="100"/></td> 
  </tr>    
</table>



In [1]:
!pip show gensim

Name: gensim
Version: 4.1.2
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /opt/anaconda3/lib/python3.8/site-packages
Requires: scipy, numpy, smart-open
Required-by: 


In [2]:
import numpy as np
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity

#carrega um modelo pré-treinado (gloVe word vectors)
word_vectors = api.load('glove-wiki-gigaword-50')

# função de word -> vec
#word2vec = lambda w: word_vectors.word_vec(w.lower())
word2vec = lambda w: word_vectors.get_vector(w.lower())

#função de similaridade
similarity = lambda x,y:cosine_similarity(x[np.newaxis,:],y[np.newaxis,:]).squeeze()

In [3]:
word_a, word_b = "father", "mother"
#word_a, word_b = "ball", "crocodile"

vec_a = word2vec(word_a) #transforma em word2vec
vec_b = word2vec(word_b) #transforma em word2vec

#calcula similaridade entre vec_a e vec_b
result = similarity(vec_a, vec_b).squeeze()
print("cosine_similarity( {} , {} ): {:.4f}".format(word_a,word_b,result))

cosine_similarity( father , mother ): 0.8909


In [4]:
pos_a, neg_a = "france" , "paris"
pos_a_v = word2vec(pos_a)
neg_a_v = word2vec(neg_a)

pos_b, neg_b = "italy", "rome"
pos_b_v = word2vec(pos_b)
neg_b_v = word2vec(neg_b)

#Quão similar são as palavras:

result = similarity(pos_a_v - neg_a_v, pos_b_v - neg_b_v)
print("cosine_similarity( {} - {} , {} - {} ): {:.4f}".format(pos_a, neg_a, pos_b, neg_b, result))

result = similarity(pos_a_v - neg_a_v, neg_b_v - pos_b_v)
print("cosine_similarity( {} - {} , {} - {} ): {:.4f}".format(pos_a, neg_a, neg_b, pos_b, result))

cosine_similarity( france - paris , italy - rome ): 0.6751
cosine_similarity( france - paris , rome - italy ): -0.6751


In [5]:
g = word2vec('king') - word2vec('man')
print(g)

[ 0.59889597  0.256      -0.42293     0.43248898 -1.04424    -0.53832996
  0.2845      0.22305998 -0.5121     -0.4179      0.031814    1.34119
  0.619771   -1.53227     0.08638     1.28       -0.63721    -0.95950997
  0.15139002  0.42474002  1.0251501  -1.40695    -0.147781   -0.59694004
  0.02067     0.58640003 -0.36815006 -1.4889     -0.73294     0.62635
 -0.6197001   0.30341998 -0.47487003  0.45971     0.17943001 -0.14885001
  0.21805    -0.243984   -1.0752699   0.73195004  0.487284    0.54300994
 -0.9561     -0.5096     -0.81159997  1.3143599  -0.43008003 -0.63135
 -1.1756101  -0.39317003]


In [6]:
#Nomes masculinos são negativos
print ('Lista de nomes e sua similaridade com o vetor g:')

name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for word in name_list:
    print("{}:{}".format(word, similarity(word2vec(word), g)))

Lista de nomes e sua similaridade com o vetor g:
john:0.19528844952583313
marie:0.026381026953458786
sophie:0.08021946251392365
ronaldo:-0.15594151616096497
priya:0.07570753246545792
rahul:0.09843342006206512
danielle:-0.1483866274356842
reza:0.11229010671377182
katy:-0.0037428135983645916
yasmin:0.13542740046977997


In [7]:
print('Outras palavras e suas similaridades:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for word in word_list:
    print("{}:{}".format(word, similarity(word2vec(word), g)))

Outras palavras e suas similaridades:
lipstick:-0.29379919171333313
guns:-0.2485954612493515
science:-0.16056984663009644
arts:-0.022254327312111855
literature:0.016476832330226898
warrior:-0.0667116791009903
doctor:-0.22291289269924164
tree:-0.16667285561561584
receptionist:-0.3380673825740814
technology:-0.09191200882196426
fashion:-0.24253956973552704
teacher:-0.2971269488334656
engineer:-0.16246071457862854
pilot:-0.24503938853740692
computer:-0.17163701355457306
singer:-0.12488631159067154


In [8]:
def complete_analogy(pos_a, neg_a, pos_b):
    
    pos_a_v, neg_a_v, pos_b_v = word2vec(pos_a), word2vec(neg_a), word2vec(pos_b)
    #calculo a diferença
    d = pos_a_v - neg_a_v
    #vou atualizando o valor no for
    max_cosine_sim = -1            
    best_word = None                   

    #for neg_b in word_vectors.vocab:   
    for neg_b in word_vectors.key_to_index: 
        
        if neg_b in [pos_a, neg_a, pos_b] :
            continue
            
        neg_b_v = word2vec(neg_b)
        #busca pos v com neg B
        cosine_sim = similarity(d , pos_b_v - neg_b_v)
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = neg_b
            
    return best_word

# as duas palavras fazem a analogia como argentina 

In [9]:
analogies_to_try = [('italy', 'italian', 'argentina'), 
                 ('india', 'delhi', 'japan'), 
                 ('man', 'woman', 'boy'), 
                 ('small', 'smaller', 'large')]

# dadas 3 palavras, encontre a próxima palavra
for analogy in analogies_to_try:
    print ('{} -> {} :: {} -> {}'.format( *analogy, complete_analogy(*analogy)))

italy -> italian :: argentina -> argentine
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger
